# Lista de exercícios 7 - MNUM-7077

#### Antonio C. da Silva Júnior

## 1 - Exercício A (10 pontos)

Matriz de custos:

In [1]:
cij = [[0, 49, 46,6, 80,8, 67,2, 35,4, 68,7, 53,6, 57,8, 73,9],
       [49, 0, 59,8, 37, 37,1, 23,4, 70,2, 48,1, 27, 41,8],
       [46,6, 59,8, 0, 70,2, 47,3, 64,8, 23, 20,1, 83,2, 52,6],
       [80,8, 37, 70,2, 0, 23,9, 60,4, 68,2, 51, 57,3, 21,3],
       [67,2, 37,1, 47,3, 23,9, 0, 57,7, 44,6, 27,5, 63,5, 6,7],
       [35,4, 23,4, 64,8, 60,4, 57,7, 0, 81,2, 60, 22,5, 63,3],
       [68,7, 70,2, 23, 68,2, 44,6, 81,2, 0, 22,1, 96,3, 47,4],
       [53,6, 48,1, 20,1, 51, 27,5, 60, 22,1, 0, 74,2, 32,6],
       [57,8, 27, 83,2, 57,3, 63,5, 22,5, 96,3, 74,2, 0, 67,5],
       [73,9, 41,8, 52,6, 21,3, 6,7, 63,3, 47,4, 32,6, 67,5, 0]]

###  1.1 - Modelo com restrições de sub-rota de Dantzig

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min z} = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$\sum\limits_{i \in Q} \sum\limits_{j \notin Q} x_{ij} \geq 1 \;\;\; \forall \; Q \subseteq \{1,...,n\}, 1 \leq |Q| \leq n-1$

$x_{ij} \in \{0,1\}$


#### 1.1.1 - Resolução

In [2]:
# Número de pontos:
n = len(cij)
print(n)

10


In [3]:
# Lógica para inclusão dinâmica das restrições de sub-rota de Dantzig:

# Combinações:
import itertools
import numpy as np

c = []
for i in range(n):
    c.append([0,1])
    
comb = np.array(list(itertools.product(*c)))
comb = np.flip(comb, axis=1)
comb = comb[1:(len(comb)-1),:] # remove todos os vazis e todos os cheios

In [4]:
# Definição dos conjuntos Q e não Q:
Q = []
nQ = []
for i in range(len(comb)):
    q = []
    n_q = []
    for j in range(len(comb[i])):
        if comb[i,j] == 1:
            q.append(j+1)
        else:
            n_q.append(j+1)
    Q.append(q)
    nQ.append(n_q)

In [5]:
import pyomo.environ as pyo

# Declaração do modelo:
modelo_D = pyo.ConcreteModel()

# Índices dos pontos:
modelo_D.M = pyo.RangeSet(n)
modelo_D.N = pyo.RangeSet(n)

# Matriz de custos:
modelo_D.c = pyo.Param(modelo_D.N, modelo_D.M, initialize=lambda modelo_D, i, j: cij[i-1][j-1])

# Variáveis de decisão:
modelo_D.x = pyo.Var(modelo_D.N,modelo_D.M, within=pyo.Binary)

# Função objetivo:
def f_objetivo(modelo):
    return sum(modelo.x[i,j] * modelo.c[i,j] for i in modelo.N for j in modelo.M)

modelo_D.objetivo = pyo.Objective(rule=f_objetivo, sense=pyo.minimize)

# Restrições

# Cada ponto só recebe de uma única origem:
def f_restr1(modelo, M):
    return sum(modelo.x[i,M] for i in modelo.N) == 1

modelo_D.restr1 = pyo.Constraint(modelo_D.M, rule=f_restr1)

# Cada ponto só envia para um destino:
def f_restr2(modelo, N):
    return sum(modelo.x[N,j] for j in modelo.M) == 1

modelo_D.restr2 = pyo.Constraint(modelo_D.N, rule=f_restr2)

# Restrições de sub-rota de Dantzig:
modelo_D.restr3 = pyo.ConstraintList()

for q, nq in zip(Q, nQ):
    modelo_D.restr3.add(sum(modelo_D.x[i,j] for i in q for j in nq) >= 1)

In [6]:
# Resolução:

# 50 execuções para obter o tempo médio:
t_D = []
for i in range(50):
    solver = pyo.SolverFactory('glpk')
    result_D = solver.solve(modelo_D)
    t_D.append(result_D.Solver.Time)

In [7]:
print(result_D)


Problem: 
- Name: unknown
  Lower bound: 79.0
  Upper bound: 79.0
  Number of objectives: 1
  Number of constraints: 1043
  Number of variables: 101
  Number of nonzeros: 23241
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.08975815773010254
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
# Trajeto:
l = list(modelo_D.x.keys())
for i in l:
    if modelo_D.x[i]() != 0:
        print(i,'--', modelo_D.x[i]())

(1, 10) -- 1.0
(2, 7) -- 1.0
(3, 5) -- 1.0
(4, 6) -- 1.0
(5, 2) -- 1.0
(6, 1) -- 1.0
(7, 4) -- 1.0
(8, 9) -- 1.0
(9, 3) -- 1.0
(10, 8) -- 1.0


In [9]:
from statistics import *

print(" Tempo médio:", mean(t_D), "\n", "Desvio padrão:", stdev(t_D))

 Tempo médio: 0.09071625232696533 
 Desvio padrão: 0.00467743146646828


###  1.2 - Modelo com restrições de sub-rota MTZ

<b>Parâmetros:</b>

$c_{ij} = \text{custo do deslocamento da origem } i \; (i = 1,...,m) \text{ para o destino } j \; (j = 1,...,n)$

<br>

<b>Variáveis de decisão:</b>

$
    x_{ij}=
    \begin{cases}
      1, & \text{se o arco } (i,j) \text{ faz parte do itinerário.} \\
      0, & \text{caso contrário}
    \end{cases}
$

<br>

<b>Formulação:</b>

$\text{min z} = \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} c_{ij} x_{ij}$

Sujeito a:

$\sum\limits_{i=1}^{n} x_{ij} = 1 \;\;\; \forall \; j=1,...,n$

$\sum\limits_{j=1}^{n} x_{ij} = 1 \;\;\; \forall \; i=1,...,m$

$x_{ij} \in \{0,1\}$

#### 1.2.1 - Resolução

In [10]:
# Número de pontos:
n = len(cij)
print(n)

10


In [11]:
import pyomo.environ as pyo

# Declaração do modelo:
modelo_MTZ = pyo.ConcreteModel()

# Índices:
modelo_MTZ.M = pyo.RangeSet(n)
modelo_MTZ.N = pyo.RangeSet(n)
modelo_MTZ.U = pyo.RangeSet(2,n)

# Matriz de custos:
modelo_MTZ.c = pyo.Param(modelo_MTZ.N, modelo_MTZ.M, initialize=lambda modelo_MTZ, i, j: cij[i-1][j-1])

# Variáveis de decisão:
modelo_MTZ.x = pyo.Var(modelo_MTZ.N,modelo_MTZ.M, within=pyo.Binary)
modelo_MTZ.u = pyo.Var(modelo_MTZ.N, within=pyo.NonNegativeIntegers,bounds=(0,n-1))

# Função objetivo:
def f_objetivo(modelo):
    return sum(modelo.x[i,j] * modelo.c[i,j] for i in modelo.N for j in modelo.M)

modelo_MTZ.objetivo = pyo.Objective(rule=f_objetivo, sense=pyo.minimize)

# Restrições

# Cada ponto só recebe de uma única origem:
def f_restr1(modelo, M):
    return sum(modelo.x[i,M] for i in modelo.N) == 1

modelo_MTZ.restr1 = pyo.Constraint(modelo_MTZ.M, rule=f_restr1)

# Cada ponto só envia para um destino:
def f_restr2(modelo, N):
    return sum(modelo.x[N,j] for j in modelo.M) == 1

modelo_MTZ.restr2 = pyo.Constraint(modelo_MTZ.N, rule=f_restr2)

# Restrições de sub-rota MTZ:
def f_restr3(modelo, i, j):
    if i!=j: 
        return modelo.u[i] - modelo.u[j] + modelo.x[i,j] * n <= n-1
    else:
        return modelo.u[i] - modelo.u[i] == 0 # sem efeito no modelo
    
modelo_MTZ.restr3 = pyo.Constraint(modelo_MTZ.U, modelo_MTZ.N, rule=f_restr3)

In [12]:
# Resolução:

# 50 execuções para obter o tempo médio:
t_MTZ = []
for i in range(50):
    solver = pyo.SolverFactory('glpk')
    result_MTZ = solver.solve(modelo_MTZ)
    t_MTZ.append(result_MTZ.Solver.Time)

In [13]:
print(result_MTZ)


Problem: 
- Name: unknown
  Lower bound: 77.0
  Upper bound: 77.0
  Number of objectives: 1
  Number of constraints: 111
  Number of variables: 111
  Number of nonzeros: 444
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 2831
      Number of created subproblems: 2831
  Error rc: 0
  Time: 0.5152127742767334
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [14]:
# Trajeto:
l = list(modelo_MTZ.x.keys())
for i in l:
    if modelo_MTZ.x[i]() != 0:
        print(i,'--', modelo_MTZ.x[i]())

(1, 10) -- 1.0
(2, 2) -- 1.0
(3, 5) -- 1.0
(4, 6) -- 1.0
(5, 4) -- 1.0
(6, 1) -- 1.0
(7, 7) -- 1.0
(8, 9) -- 1.0
(9, 3) -- 1.0
(10, 8) -- 1.0


In [15]:
from statistics import *

print(" Tempo médio:", mean(t_MTZ), "\n", "Desvio padrão:", stdev(t_MTZ))

 Tempo médio: 0.5017868280410767 
 Desvio padrão: 0.00955160443318802
